In [38]:
import random
import operator
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    msg1: str
    msg2: str
    final: str

graph_builder = StateGraph(State)

In [39]:
import json
from pydantic import BaseModel

class TA(BaseModel):
    a: str

class TB(BaseModel):
    b: str
    c: str

class TEMP(BaseModel):
    ta: TA
    tb: TB

temp = TEMP.model_validate({"ta": {"a": "aaa"}, "tb": {"b": "bbb", "c": "ccc"}})
temp

TEMP(ta=TA(a='aaa'), tb=TB(b='bbb', c='ccc'))

In [40]:
def init(state: State, config):
    print(config['configurable'].get('key'))
    return state

def node1(state: State, config):
    msg1 = state.get('msg1',"")
    msg1 = "store node 1"
    temp = config['configurable'].get("temp")
    print(temp.ta.a)
    # return State(msg1=msg1)
    return {"msg1": msg1}

def node2(state: State, config):
    msg2 = state.get("msg2","")
    msg2 = "store node 2"
    temp = config['configurable'].get("temp")
    print(temp.tb.b)
    # return State(msg2=msg2)
    return {"msg2": msg2}

def node3(state: State, config):
    print(config['configurable'].get('key'))
    print(state)
    temp = config['configurable'].get("temp")
    print(temp.tb.c)
    msg1 = state.get("msg1")
    msg2 = state.get("msg2")
    final = f'{msg1} | {msg2}'
    # return State(final=final)
    return {"final": final}

graph_builder.add_node("init",init)
graph_builder.add_node("node1",node1)
graph_builder.add_node("node2",node2)
graph_builder.add_node("node3",node3)

graph_builder.set_entry_point("init")

graph_builder.add_edge("init","node1")
graph_builder.add_edge("init","node2")
graph_builder.add_edge("node1","node3")
graph_builder.add_edge("node2","node3")
graph_builder.add_edge("node3",END)

graph = graph_builder.compile()

In [41]:
graph.invoke({"msg1":""}, {'key':'value', "temp": temp})

value
aaa
bbb
value
{'msg1': 'store node 1', 'msg2': 'store node 2'}
ccc


{'msg1': 'store node 1',
 'msg2': 'store node 2',
 'final': 'store node 1 | store node 2'}